# Azure Cognitive Search Integrated Vectorization Sample
This code demonstrates how to use Azure Cognitive Search as a Vector store by automatically chunking and generating embeddings using the Azure OpenAI Embedding skill as part of the skillset pipeline in Azure Cognitive Search. 
## Prerequisites
To run the code, install the following packages. This was tested with Python v3.11.5

In [ ]:
! pip install requirements.txt --quiet

## Import required libraries and environment variables

In [2]:
# Import required libraries  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient  
from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,

    QueryType,
    VectorizedQuery,
    
    VectorFilterMode,    
)
#from azure.search.documents.models import (
   # RawVectorQuery,
 #   VectorizedQuery,
   # QueryLanguage,
#)
from azure.search.documents.indexes.models import (  
    #AzureOpenAIParameters,  
    #AzureOpenAIVectorizer,  
    ExhaustiveKnnParameters,  
    ExhaustiveKnnAlgorithmConfiguration,
    FieldMapping,  
    HnswParameters,  
    #HnswVectorSearchAlgorithmConfiguration,
    HnswAlgorithmConfiguration,  
    #IndexProjectionMode,  
    InputFieldMappingEntry,  
    OutputFieldMappingEntry,  
    SemanticPrioritizedFields,    
    SearchField,  
    SearchFieldDataType,  
    SearchIndex,  
    SearchIndexer,  
    SearchIndexerDataContainer,  
    SearchIndexerDataSourceConnection,  
    #SearchIndexerIndexProjectionSelector,  
    #SearchIndexerIndexProjections,  
    #SearchIndexerIndexProjectionsParameters,  
    SearchIndexerSkillset,  
    SemanticConfiguration,  
    SemanticField,  
    SemanticSearch,  
    SplitSkill,
    SqlIntegratedChangeTrackingPolicy,  
    VectorSearch,  
    VectorSearchAlgorithmKind,  
    VectorSearchAlgorithmMetric,  
    VectorSearchProfile,  
)  
#from azure.storage.blob import BlobServiceClient  
import openai
#import tiktoken

#from openai.embeddings_utils import get_embedding
from dotenv import load_dotenv  
import os  
#from openai import AzureOpenAIEmbeddingSkill
  
# Configure environment variables
load_dotenv()  
service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")  
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME_01")  
key = os.getenv("AZURE_SEARCH_ADMIN_KEY")  
openai.api_type = "azure"  
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")  
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")  
openai.api_version = os.getenv("AZURE_OPENAI_API_VERSION")  
model: str = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL")
blob_connection_string = os.getenv("BLOB_CONNECTION_STRING")  
container_name = os.getenv("BLOB_CONTAINER_NAME")  
credential = AzureKeyCredential(key)  
EMBEDDING_LENGTH = 1536     # 768 for GPT-2, 1024 for GPT-3
server = os.getenv("MYSQL_SERVER")
database = os.getenv("MYSQL_DATABASE")
username = os.getenv("MYSQL_USERNAME")
password = os.getenv("MYSQL_PASSWORD")
table_name = os.getenv("MYSQL_TABLE_NAME")

## Connect to MySQL to a data source in Cognitive Search

In [ ]:
ds_conn_str = f'Server={server}; Port=3306; Database={database}; Uid={username}; Pwd={password}; SslMode=Preferred;'

# Create a data source
ds_client = SearchIndexerClient(service_endpoint, AzureKeyCredential(key))
container = SearchIndexerDataContainer(name=table_name)

#change_detection_policy = SqlIntegratedChangeTrackingPolicy()

data_source_connection = SearchIndexerDataSourceConnection(
    name=f"{index_name}-ds",
    type="mysql",
    connection_string=ds_conn_str,
    container=container
    #,data_change_detection_policy=change_detection_policy
)
data_source = ds_client.create_or_update_data_source_connection(data_source_connection)

print(f"Data source '{data_source.name}' created or updated")

## Create a search index

In [27]:
# Create a search index  
index_client = SearchIndexClient(endpoint=service_endpoint, credential=credential)  
fields = [  
  # Properties of individual chunk
    SearchField(name="Id", type=SearchFieldDataType.String, key=True,
                sortable=True, filterable=True, facetable=True, analyzer_name="keyword"),
    SearchField(name="chunk", type=SearchFieldDataType.String, sortable=False, filterable=False, facetable=False),
    SearchField(name="vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), 
                vector_search_dimensions=EMBEDDING_LENGTH, vector_search_profile_name="myHnswProfile"),
    # Properties of original row in DB that the chunk belonged to
    SearchField(name="parent_id", type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=True),
    SearchField(name="parent_numero", type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=True),
    SearchField(name="parent_descripcion_breve", type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=True),
    SearchField(name="parent_descripcion", type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=True)

]  


In [32]:
# Configure the vector search configuration  
vector_search = VectorSearch(  
    algorithms=[  
        HnswAlgorithmConfiguration(  
            name="myHnsw",  
            kind=VectorSearchAlgorithmKind.HNSW,  
            parameters=HnswParameters(  
                m=4,  
                ef_construction=400,  
                ef_search=500,  
                metric=VectorSearchAlgorithmMetric.COSINE,  
            ),  
        ),  
        ExhaustiveKnnAlgorithmConfiguration(  
            name="myExhaustiveKnn",  
            kind=VectorSearchAlgorithmKind.EXHAUSTIVE_KNN,  
            parameters=ExhaustiveKnnParameters(  
                metric=VectorSearchAlgorithmMetric.COSINE,  
            ),  
        ),  
    ],  
    profiles=[  
        VectorSearchProfile(  
            name="myHnswProfile",  
            algorithm_configuration_name="myHnsw",  
           # vectorizer="myOpenAI",  
        ),  
        VectorSearchProfile(  
            name="myExhaustiveKnnProfile",  
            algorithm_configuration_name="myExhaustiveKnn",  
            #vectorizer="myOpenAI",  
        ),  
    ],  
    
)  
  


To add the vectorizer to the index you will need the version `11.4.0b12` of [azure-search-documents] which is a pre-release version. It's better to created it directly on the Azure Portal and do the reference to the Hnsw algorithm

In [ ]:
vectorizers=[  
        AzureOpenAIVectorizer(  
            name="myOpenAI",  
            kind="azureOpenAI",  
            azure_open_ai_parameters=AzureOpenAIParameters(  
                resource_uri=os.getenv("AZURE_OPENAI_ENDPOINT"),  
                deployment_id=model,  
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
            ),  
        ),  
    ],  

In [35]:
semantic_config = SemanticConfiguration(  
    name="my-semantic-config",  
    prioritized_fields=SemanticPrioritizedFields(  
        content_fields=[SemanticField(field_name="Id")]  
    ),  
)  
  


In [37]:
# Create the semantic settings with the configuration  
semantic_search = SemanticSearch(configurations=[semantic_config])  
  
# Create the search index with the semantic settings  
index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search, semantic_search=semantic_search)  
result = index_client.create_or_update_index(index)  
print(f"{result.name} created")  

ticket-mysql-vector-02 created


## Create a skillset

To create a skillset you will need the version `11.4.0b12` of [azure-search-documents] which is a pre-release version. 

In [ ]:
! pip install ../whl/azure_search_documents-11.4.0b12-py3-none-any.whl --quiet  

In [5]:
# Create a skillset  
skillset_name = f"{index_name}-skillset"  

In [ ]:
split_skill = SplitSkill(  
    description="Split skill to chunk documents",  
    text_split_mode="pages",  
    context="/document",  
    maximum_page_length=300,  
    page_overlap_length=20,  
    inputs=[  
        InputFieldMappingEntry(name="text", source="/document/descripcion"),  
    ],  
    outputs=[  
        OutputFieldMappingEntry(name="textItems", target_name="pages")  
    ],  
)  
  
embedding_skill = AzureOpenAIEmbeddingSkill(  
    description="Skill to generate embeddings via Azure OpenAI",  
    context="/document/pages/*",  
    resource_uri=os.getenv("AZURE_OPENAI_ENDPOINT"),  
    deployment_id=model,  
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    inputs=[  
        InputFieldMappingEntry(name="text", source="/document/pages/*"),  
    ],  
    outputs=[  
        OutputFieldMappingEntry(name="embedding", target_name="vector")  
    ],  
)  
  
index_projections = SearchIndexerIndexProjections(  
    selectors=[  
        SearchIndexerIndexProjectionSelector(  
            target_index_name=index_name,  
            parent_key_field_name="parent_id",  
            source_context="/document/pages/*",  
            mappings=[  
                InputFieldMappingEntry(name="chunk", source="/document/pages/*"),
                InputFieldMappingEntry(name="vector", source="/document/pages/*/vector"),
               # InputFieldMappingEntry(name="Id", source="/document/numero"),
                InputFieldMappingEntry(name="parent_numero", source="/document/numero"),
                InputFieldMappingEntry(name="parent_descripcion_breve", source="/document/descripcion_breve"),
                InputFieldMappingEntry(name="parent_descripcion", source="/document/descripcion")
            ],  
        ),  
    ],  
    parameters=SearchIndexerIndexProjectionsParameters(  
        projection_mode=IndexProjectionMode.SKIP_INDEXING_PARENT_DOCUMENTS  
    ),  
)  
  
skillset = SearchIndexerSkillset(  
    name=skillset_name,  
    description="Skillset to chunk documents and generating embeddings",  
    skills=[split_skill, embedding_skill],  
    index_projections=index_projections,  
)  
  
client = SearchIndexerClient(service_endpoint, AzureKeyCredential(key))  
client.create_or_update_skillset(skillset)  
print(f"{skillset.name} created")  


## Create an indexer

In [7]:
# Create an indexer  
indexer_name = f"{index_name}-indexer"  
  
indexer = SearchIndexer(  
    name=indexer_name,  
    description="Indexer to index documents and generate embeddings",  
    skillset_name=skillset_name,  
    target_index_name=index_name,  
    #data_source_name=data_source.name,  
    data_source_name="ticket-mysql-ds",
    # Map the metadata_storage_name field to the title field in the index to display the PDF title in the search results  
    field_mappings=[FieldMapping(source_field_name="numero", target_field_name="Id")]  
)  
  
indexer_client = SearchIndexerClient(service_endpoint, AzureKeyCredential(key))  
indexer_result = indexer_client.create_or_update_indexer(indexer)  
  
# Run the indexer  
indexer_client.run_indexer(indexer_name)  
print(f' {indexer_name} created')  


 ticket-mysql-vector-02-indexer created


## Perform a vector similarity search

This example shows a pure vector search using the vectorizable text query, all you need to do is pass in text and your vectorizer will handle the query vectorization.

In [8]:
#from openai import OpenA
from anyio import sleep
#get embeddings from azureopenai of a text 
def get_embeddings(text: str):
    # There are a few ways to get embeddings. This is just one example.
    # show me thee code generate embeddings from OpenAI from a text
    
    client = openai.AzureOpenAI(
        azure_endpoint=openai.api_base,
        api_key=openai.api_key,
        api_version=openai.api_version,
    )
    text = text.replace("\n", " ")
    count=0
    while True:
        try:
            embedding = client.embeddings.create(input=[text], model=model)
            break;
        except openai.error.RateLimitError:
            count+=1
            print(f'RateLimitError Count: {count}')
            sleep(2)
    return embedding.data[0].embedding



In [9]:
#from openai.embeddings_utils import get_embedding,
# Pure Vector Search
query = "que se requiere para liberar la app afore?"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector_query = VectorizedQuery(vector=get_embeddings(query), fields="vector", exhaustive=True, k_nearest_neighbors=3)
# Use the below query to pass in the raw vector query instead of the query vectorization
# vector_query = RawVectorQuery(vector=generate_embeddings(query), k=3, fields="vector")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["parent_id", "Id", "chunk"],
    top=1
)  
  
for result in results:  
    print(f"parent_id: {result['parent_id']}")  
    print(f"chunk_id: {result['Id']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['chunk']}")   


parent_id: CHG0032603
chunk_id: cd832933ff02_CHG0032603_pages_0
Score: 0.8629935
Content: SE REQUIERE LIBERAR LA APP DE MODIFICACION DE DATOS  EN ATENCION A FT RITM0123459 PARA LA CORRECCION DE LAS SIGUIENTES INCIDENCIAS.

*SE ENCUENTRAN CON EL CAMPO VACÍO DE PAÍS EN LOS DATOS DE DOMICILIO PARTICULAR Y LABORAL.

*TRAMITES DE APODERADO LEGAL LLEGABAN COMO BENEFICIARIO A PORTAL MIT


## Perform a hybrid search

In [10]:
# Hybrid Search
query = "que se requiere para liberar la app afore?"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector_query = VectorizedQuery(vector=get_embeddings(query), fields="vector", exhaustive=True, k_nearest_neighbors=3)
  
results = search_client.search(  
    search_text=query,  
    vector_queries= [vector_query],
    select=["parent_id", "Id", "chunk"],
    top=1
)  
  
for result in results:  
    print(f"parent_id: {result['parent_id']}")  
    print(f"chunk_id: {result['Id']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['chunk']}")  


parent_id: CHG0032603
chunk_id: cd832933ff02_CHG0032603_pages_0
Score: 0.03333333507180214
Content: SE REQUIERE LIBERAR LA APP DE MODIFICACION DE DATOS  EN ATENCION A FT RITM0123459 PARA LA CORRECCION DE LAS SIGUIENTES INCIDENCIAS.

*SE ENCUENTRAN CON EL CAMPO VACÍO DE PAÍS EN LOS DATOS DE DOMICILIO PARTICULAR Y LABORAL.

*TRAMITES DE APODERADO LEGAL LLEGABAN COMO BENEFICIARIO A PORTAL MIT


## Perform a hybrid search + semantic reranking

In [11]:
from azure.search.documents.models import QueryType

# Semantic Hybrid Search
query = "que se requiere para liberar la app afore?"

search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))
vector_query = VectorizedQuery(vector=get_embeddings(query), fields="vector", exhaustive=True, k_nearest_neighbors=3)

results = search_client.search(  
    search_text=query,
    vector_queries=[vector_query],
    select=["parent_id", "Id", "chunk"],
    #query_type=QueryType.SEMANTIC, queryL="en-us", semantic_configuration_name='my-semantic-config', query_caption=QueryCaptionType.EXTRACTIVE, query_answer=QueryAnswerType.EXTRACTIVE,
    query_type=QueryType.SEMANTIC, semantic_configuration_name='my-semantic-config', query_caption=QueryCaptionType.EXTRACTIVE, query_answer=QueryAnswerType.EXTRACTIVE,
    top=2
)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    print(f"parent_id: {result['parent_id']}")  
    print(f"chunk_id: {result['Id']}")  
    print(f"Reranker Score: {result['@search.reranker_score']}")
    print(f"Content: {result['chunk']}")  

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")


parent_id: CHG0032603
chunk_id: cd832933ff02_CHG0032603_pages_0
Reranker Score: 1.6374846696853638
Content: SE REQUIERE LIBERAR LA APP DE MODIFICACION DE DATOS  EN ATENCION A FT RITM0123459 PARA LA CORRECCION DE LAS SIGUIENTES INCIDENCIAS.

*SE ENCUENTRAN CON EL CAMPO VACÍO DE PAÍS EN LOS DATOS DE DOMICILIO PARTICULAR Y LABORAL.

*TRAMITES DE APODERADO LEGAL LLEGABAN COMO BENEFICIARIO A PORTAL MIT
Caption: 

parent_id: CHG0032599
chunk_id: 33429450d544_CHG0032599_pages_0
Reranker Score: 1.6374846696853638
Content: SE REQUIERE LA ACTUALIZACIÓN DE LOS COMPONENTES DE ACTIVIDAD COMERCIAL, ESTUDIO DE RETIRO PERSONALIZADO Y PROSPECTA QUE INCLUYE LAS MEJORAS DE CORRECCIÓN DE HORARIO EN LA ZONA CST Y CAMBIO DE SQL LITE A SECURE STORAGE, MODIFICACIÓN IOS MENÚ PRINCIPAL.
Caption: 

